In [1]:
# Import library-library
import os
import glob
from tqdm import tqdm
import numpy as np

# Data Preparation and Preprocessing
import pandas as pd
import re
from string import digits

# Word Embedding
import joblib
from keybert import KeyBERT
kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')

# Input and Expansion Query
import nltk
nltk.download('punkt')
import math
from textblob import TextBlob
from yake import KeywordExtractor
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk import tokenize
from operator import itemgetter
from nltk.tokenize import word_tokenize
from nltk.tag import CRFTagger
from textblob import TextBlob
NLTK_StopWords = stopwords.words('indonesian')

d:\HERRI\Tugas Akhir\pra\geographical_news_scrapper\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
excluded_words = ["tempat", "waktu", "hari"]

NLTK_StopWords = stopwords.words('indonesian')
NLTK_StopWords.extend(["detik", "detikjatim", "detikjateng", "detikjabar", "detiksulsel", "detiksumbar", "detikbali", "detikpapua", "detiksulteng", "detikmaluku", "detjatim", "detikcom", "allahumma", "aamiin", "allah", "bismillah"])
NLTK_StopWords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
# txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
# NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
NLTK_StopWords = set(NLTK_StopWords)

In [3]:
def preprocessingCorpus(berita):
    s = str(berita)
    s = s.lower()
    s = s.replace('\n', ' ')
    s = s.replace('\r', ' ')
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]
    T = [t for t in tokens if ((t in excluded_words) or (t not in NLTK_StopWords))]
    return T

ct =CRFTagger()
ct.set_model_file('all_indo_man_tag_corpus_model.crf.tagger')
def preprocessingKeparahan(berita):
    s = berita
    # s = s.lower()
    tokens = word_tokenize(s)
    return tokens

In [4]:
kta="olahraga fengan sukamu"
print(preprocessingCorpus(kta))

['olahraga', 'fengan', 'sukamu']


In [5]:

df_test = pd.read_csv('df_test.csv')
# df_test = pd.read_csv('df_test_nopre.csv')
df_test = df_test[pd.notnull(df_test['description'])]
print(df_test.info())
print ('-'*90)
# document_text_test= joblib.load('document_text_test.pkl')
print(len(df_test))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243 entries, 0 to 242
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        243 non-null    object
 1   date         243 non-null    object
 2   description  243 non-null    object
 3   source       243 non-null    object
dtypes: object(4)
memory usage: 9.5+ KB
None
------------------------------------------------------------------------------------------
243


In [6]:
df_train = pd.read_csv('df_train.csv')
df_train = df_train[pd.notnull(df_train['description'])]
print(df_train.info())
print ('-'*90)
# document_text_train= joblib.load('document_text_train.pkl')
print(len(df_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2178 entries, 0 to 2178
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        2178 non-null   object
 1   date         2178 non-null   object
 2   description  2178 non-null   object
 3   source       2178 non-null   object
dtypes: object(4)
memory usage: 85.1+ KB
None
------------------------------------------------------------------------------------------
2178


In [7]:
def struktur1(arr,indeks):
    
    if(indeks+1<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='CD' or arr[0][indeks+1][0]=='Seorang'  or arr[0][indeks+1][0]=='seorang'  or arr[0][indeks+1][0]=='seseorang' or arr[0][indeks+1][0]=='Seseorang' or arr[0][indeks+1][0]=='Empat' or arr[0][indeks+1][0]=='Lima' or arr[0][indeks+1][0]=='Enam' or arr[0][indeks+1][0]=='Tujuh' or arr[0][indeks+1][0]=='Delapan' or arr[0][indeks+1][0]=='Sembilan' or arr[0][indeks+1][0]=='Sepuluh')):
        # print('masuk1')
        return arr[0][indeks+1][0]
    elif(indeks-1>=0 
        and (arr[0][indeks-1][1]=='CD' or arr[0][indeks-1][0]=='Seorang'  or arr[0][indeks-1][0]=='seorang'  or arr[0][indeks-1][0]=='seseorang' or arr[0][indeks-1][0]=='Seseorang' or arr[0][indeks-1][0]=='Empat' or arr[0][indeks-1][0]=='Lima' or arr[0][indeks-1][0]=='Enam' or arr[0][indeks-1][0]=='Tujuh' or arr[0][indeks-1][0]=='Delapan' or arr[0][indeks-1][0]=='Sembilan' or arr[0][indeks-1][0]=='Sepuluh')):
        # print('masuk12')
        return arr[0][indeks-1][0]
    elif(indeks+2<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB') 
        and (arr[0][indeks+2][1]=='CD' or arr[0][indeks+2][0]=='Seorang'  or arr[0][indeks+2][0]=='seorang'  or arr[0][indeks+2][0]=='seseorang' or arr[0][indeks+2][0]=='Seseorang' or arr[0][indeks+2][0]=='Empat' or arr[0][indeks+2][0]=='Lima' or arr[0][indeks+2][0]=='Enam' or arr[0][indeks+2][0]=='Tujuh' or arr[0][indeks+2][0]=='Delapan' or arr[0][indeks+2][0]=='Sembilan' or arr[0][indeks+2][0]=='Sepuluh')):
        return arr[0][indeks+2][0]
    elif(indeks-2>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB') 
        and (arr[0][indeks-2][1]=='CD' or arr[0][indeks-2][0]=='Seorang'  or arr[0][indeks-2][0]=='seorang'  or arr[0][indeks-2][0]=='seseorang' or arr[0][indeks-2][0]=='Seseorang' or arr[0][indeks-2][0]=='Empat' or arr[0][indeks-2][0]=='Lima' or arr[0][indeks-2][0]=='Enam' or arr[0][indeks-2][0]=='Tujuh' or arr[0][indeks-2][0]=='Delapan' or arr[0][indeks-2][0]=='Sembilan' or arr[0][indeks-2][0]=='Sepuluh')):
        return arr[0][indeks-2][0]
    elif(indeks+3<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='CD' or arr[0][indeks+3][0]=='Seorang'  or arr[0][indeks+3][0]=='seorang'  or arr[0][indeks+3][0]=='seseorang' or arr[0][indeks+3][0]=='Seseorang' or arr[0][indeks+3][0]=='Empat' or arr[0][indeks+3][0]=='Lima' or arr[0][indeks+3][0]=='Enam' or arr[0][indeks+3][0]=='Tujuh' or arr[0][indeks+3][0]=='Delapan' or arr[0][indeks+3][0]=='Sembilan' or arr[0][indeks+3][0]=='Sepuluh')):
        # print('masuk14')
        return arr[0][indeks+3][0]
    elif(indeks-3>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='CD' or arr[0][indeks-3][0]=='Seorang'  or arr[0][indeks-3][0]=='seorang'  or arr[0][indeks-3][0]=='seseorang' or arr[0][indeks-3][0]=='Seseorang' or arr[0][indeks-3][0]=='Empat' or arr[0][indeks-3][0]=='Lima' or arr[0][indeks-3][0]=='Enam' or arr[0][indeks-3][0]=='Tujuh' or arr[0][indeks-3][0]=='Delapan' or arr[0][indeks-3][0]=='Sembilan' or arr[0][indeks-3][0]=='Sepuluh')):
        # print('masuk15')
        return arr[0][indeks-3][0]
    elif(indeks+5<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='CD' or arr[0][indeks+4][0]=='Seorang'  or arr[0][indeks+4][0]=='seorang'  or arr[0][indeks+4][0]=='seseorang' or arr[0][indeks+4][0]=='Seseorang' or arr[0][indeks+4][0]=='Empat' or arr[0][indeks+4][0]=='Lima' or arr[0][indeks+4][0]=='Enam' or arr[0][indeks+4][0]=='Tujuh' or arr[0][indeks+4][0]=='Delapan' or arr[0][indeks+4][0]=='Sembilan' or arr[0][indeks+4][0]=='Sepuluh')):
        return arr[0][indeks+4][0]
    elif(indeks-4>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='CD' or arr[0][indeks-4][0]=='Seorang'  or arr[0][indeks-4][0]=='seorang'  or arr[0][indeks-4][0]=='seseorang' or arr[0][indeks-4][0]=='Seseorang' or arr[0][indeks-4][0]=='Empat' or arr[0][indeks-4][0]=='Lima' or arr[0][indeks-4][0]=='Enam' or arr[0][indeks-4][0]=='Tujuh' or arr[0][indeks-4][0]=='Delapan' or arr[0][indeks-4][0]=='Sembilan' or arr[0][indeks-4][0]=='Sepuluh')):
        return arr[0][indeks-4][0]
    elif(indeks+5<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='CD' or arr[0][indeks+5][0]=='Seorang'  or arr[0][indeks+5][0]=='seorang'  or arr[0][indeks+5][0]=='seseorang' or arr[0][indeks+5][0]=='Seseorang' or arr[0][indeks+5][0]=='Empat' or arr[0][indeks+5][0]=='Lima' or arr[0][indeks+5][0]=='Enam' or arr[0][indeks+5][0]=='Tujuh' or arr[0][indeks+5][0]=='Delapan' or arr[0][indeks+5][0]=='Sembilan' or arr[0][indeks+5][0]=='Sepuluh')):
        return arr[0][indeks+5][0]
    elif(indeks-5>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='CD' or arr[0][indeks-5][0]=='Seorang'  or arr[0][indeks-5][0]=='seorang'  or arr[0][indeks-5][0]=='seseorang' or arr[0][indeks-5][0]=='Seseorang' or arr[0][indeks-5][0]=='Empat' or arr[0][indeks-5][0]=='Lima' or arr[0][indeks-5][0]=='Enam' or arr[0][indeks-5][0]=='Tujuh' or arr[0][indeks-5][0]=='Delapan' or arr[0][indeks-5][0]=='Sembilan' or arr[0][indeks-5][0]=='Sepuluh')):
        return arr[0][indeks-5][0]
    elif(indeks+6<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='CD' or arr[0][indeks+6][0]=='Seorang'  or arr[0][indeks+6][0]=='seorang'  or arr[0][indeks+6][0]=='seseorang' or arr[0][indeks+6][0]=='Seseorang' or arr[0][indeks+6][0]=='Empat' or arr[0][indeks+6][0]=='Lima' or arr[0][indeks+6][0]=='Enam' or arr[0][indeks+6][0]=='Tujuh' or arr[0][indeks+6][0]=='Delapan' or arr[0][indeks+6][0]=='Sembilan' or arr[0][indeks+6][0]=='Sepuluh')):
        return arr[0][indeks+6][0]
    elif(indeks-6>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='CD' or arr[0][indeks-6][0]=='Seorang'  or arr[0][indeks-6][0]=='seorang'  or arr[0][indeks-6][0]=='seseorang' or arr[0][indeks-6][0]=='Seseorang' or arr[0][indeks-6][0]=='Empat' or arr[0][indeks-6][0]=='Lima' or arr[0][indeks-6][0]=='Enam' or arr[0][indeks-6][0]=='Tujuh' or arr[0][indeks-6][0]=='Delapan' or arr[0][indeks-6][0]=='Sembilan' or arr[0][indeks-6][0]=='Sepuluh')):
        return arr[0][indeks-6][0]
    elif(indeks+7<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='VB' or arr[0][indeks+6][1]=='IN' or arr[0][indeks+6][1]=='NN' or arr[0][indeks+6][1]=='NNP' or arr[0][indeks+6][1]=='SC' or arr[0][indeks+6][1]=='JJ' or arr[0][indeks+6][1]=='PR'  or arr[0][indeks+6][1]=='NND'  or arr[0][indeks+6][1]=='RB') 
        and (arr[0][indeks+7][1]=='CD' or arr[0][indeks+7][0]=='Seorang'  or arr[0][indeks+7][0]=='seorang'  or arr[0][indeks+7][0]=='seseorang' or arr[0][indeks+7][0]=='Seseorang' or arr[0][indeks+7][0]=='Empat' or arr[0][indeks+7][0]=='Lima' or arr[0][indeks+7][0]=='Enam' or arr[0][indeks+7][0]=='Tujuh' or arr[0][indeks+7][0]=='Delapan' or arr[0][indeks+7][0]=='Sembilan' or arr[0][indeks+7][0]=='Sepuluh')):
        return arr[0][indeks+7][0]
    elif(indeks-7>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='VB' or arr[0][indeks-6][1]=='IN' or arr[0][indeks-6][1]=='NN' or arr[0][indeks-6][1]=='NNP' or arr[0][indeks-6][1]=='SC' or arr[0][indeks-6][1]=='JJ' or arr[0][indeks-6][1]=='PR'  or arr[0][indeks-6][1]=='NND'  or arr[0][indeks-6][1]=='RB') 
        and (arr[0][indeks-7][1]=='CD' or arr[0][indeks-7][0]=='Seorang'  or arr[0][indeks-7][0]=='seorang'  or arr[0][indeks-7][0]=='seseorang' or arr[0][indeks-7][0]=='Seseorang' or arr[0][indeks-7][0]=='Empat' or arr[0][indeks-7][0]=='Lima' or arr[0][indeks-7][0]=='Enam' or arr[0][indeks-7][0]=='Tujuh' or arr[0][indeks-7][0]=='Delapan' or arr[0][indeks-7][0]=='Sembilan' or arr[0][indeks-7][0]=='Sepuluh')):
        return arr[0][indeks-7][0]
    elif(indeks+8<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='VB' or arr[0][indeks+6][1]=='IN' or arr[0][indeks+6][1]=='NN' or arr[0][indeks+6][1]=='NNP' or arr[0][indeks+6][1]=='SC' or arr[0][indeks+6][1]=='JJ' or arr[0][indeks+6][1]=='PR'  or arr[0][indeks+6][1]=='NND'  or arr[0][indeks+6][1]=='RB') 
        and (arr[0][indeks+7][1]=='VB' or arr[0][indeks+7][1]=='IN' or arr[0][indeks+7][1]=='NN' or arr[0][indeks+7][1]=='NNP' or arr[0][indeks+7][1]=='SC' or arr[0][indeks+7][1]=='JJ' or arr[0][indeks+7][1]=='PR'  or arr[0][indeks+7][1]=='NND'  or arr[0][indeks+7][1]=='RB') 
        and (arr[0][indeks+8][1]=='CD' or arr[0][indeks+8][0]=='Seorang'  or arr[0][indeks+8][0]=='seorang'  or arr[0][indeks+8][0]=='seseorang' or arr[0][indeks+8][0]=='Seseorang' or arr[0][indeks+8][0]=='Empat' or arr[0][indeks+8][0]=='Lima' or arr[0][indeks+8][0]=='Enam' or arr[0][indeks+8][0]=='Tujuh' or arr[0][indeks+8][0]=='Delapan' or arr[0][indeks+8][0]=='Sembilan' or arr[0][indeks+8][0]=='Sepuluh')):
        return arr[0][indeks+8][0]
    elif(indeks-8>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='VB' or arr[0][indeks-6][1]=='IN' or arr[0][indeks-6][1]=='NN' or arr[0][indeks-6][1]=='NNP' or arr[0][indeks-6][1]=='SC' or arr[0][indeks-6][1]=='JJ' or arr[0][indeks-6][1]=='PR'  or arr[0][indeks-6][1]=='NND'  or arr[0][indeks-6][1]=='RB') 
        and (arr[0][indeks-7][1]=='VB' or arr[0][indeks-7][1]=='IN' or arr[0][indeks-7][1]=='NN' or arr[0][indeks-7][1]=='NNP' or arr[0][indeks-7][1]=='SC' or arr[0][indeks-7][1]=='JJ' or arr[0][indeks-7][1]=='PR'  or arr[0][indeks-7][1]=='NND'  or arr[0][indeks-7][1]=='RB') 
        and (arr[0][indeks-8][1]=='CD' or arr[0][indeks-8][0]=='Seorang'  or arr[0][indeks-8][0]=='seorang'  or arr[0][indeks-8][0]=='seseorang' or arr[0][indeks-8][0]=='Seseorang' or arr[0][indeks-8][0]=='Empat' or arr[0][indeks-8][0]=='Lima' or arr[0][indeks-8][0]=='Enam' or arr[0][indeks-8][0]=='Tujuh' or arr[0][indeks-8][0]=='Delapan' or arr[0][indeks-8][0]=='Sembilan' or arr[0][indeks-8][0]=='Sepuluh')):
        return arr[0][indeks-8][0]
    elif(indeks+9<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='VB' or arr[0][indeks+6][1]=='IN' or arr[0][indeks+6][1]=='NN' or arr[0][indeks+6][1]=='NNP' or arr[0][indeks+6][1]=='SC' or arr[0][indeks+6][1]=='JJ' or arr[0][indeks+6][1]=='PR'  or arr[0][indeks+6][1]=='NND'  or arr[0][indeks+6][1]=='RB') 
        and (arr[0][indeks+7][1]=='VB' or arr[0][indeks+7][1]=='IN' or arr[0][indeks+7][1]=='NN' or arr[0][indeks+7][1]=='NNP' or arr[0][indeks+7][1]=='SC' or arr[0][indeks+7][1]=='JJ' or arr[0][indeks+7][1]=='PR'  or arr[0][indeks+7][1]=='NND'  or arr[0][indeks+7][1]=='RB') 
        and (arr[0][indeks+8][1]=='VB' or arr[0][indeks+8][1]=='IN' or arr[0][indeks+8][1]=='NN' or arr[0][indeks+8][1]=='NNP' or arr[0][indeks+8][1]=='SC' or arr[0][indeks+8][1]=='JJ' or arr[0][indeks+8][1]=='PR'  or arr[0][indeks+8][1]=='NND'  or arr[0][indeks+8][1]=='RB') 
        and (arr[0][indeks+9][1]=='CD' or arr[0][indeks+9][0]=='Seorang'  or arr[0][indeks+9][0]=='seorang'  or arr[0][indeks+9][0]=='seseorang' or arr[0][indeks+9][0]=='Seseorang' or arr[0][indeks+9][0]=='Empat' or arr[0][indeks+9][0]=='Lima' or arr[0][indeks+9][0]=='Enam' or arr[0][indeks+9][0]=='Tujuh' or arr[0][indeks+9][0]=='Delapan' or arr[0][indeks+9][0]=='Sembilan' or arr[0][indeks+9][0]=='Sepuluh')):
        return arr[0][indeks+9][0]
    elif(indeks-9>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='VB' or arr[0][indeks-6][1]=='IN' or arr[0][indeks-6][1]=='NN' or arr[0][indeks-6][1]=='NNP' or arr[0][indeks-6][1]=='SC' or arr[0][indeks-6][1]=='JJ' or arr[0][indeks-6][1]=='PR'  or arr[0][indeks-6][1]=='NND'  or arr[0][indeks-6][1]=='RB') 
        and (arr[0][indeks-7][1]=='VB' or arr[0][indeks-7][1]=='IN' or arr[0][indeks-7][1]=='NN' or arr[0][indeks-7][1]=='NNP' or arr[0][indeks-7][1]=='SC' or arr[0][indeks-7][1]=='JJ' or arr[0][indeks-7][1]=='PR'  or arr[0][indeks-7][1]=='NND'  or arr[0][indeks-7][1]=='RB') 
        and (arr[0][indeks-8][1]=='VB' or arr[0][indeks-8][1]=='IN' or arr[0][indeks-8][1]=='NN' or arr[0][indeks-8][1]=='NNP' or arr[0][indeks-8][1]=='SC' or arr[0][indeks-8][1]=='JJ' or arr[0][indeks-8][1]=='PR'  or arr[0][indeks-8][1]=='NND'  or arr[0][indeks-8][1]=='RB') 
        and (arr[0][indeks-9][1]=='CD' or arr[0][indeks-9][0]=='Seorang'  or arr[0][indeks-9][0]=='seorang'  or arr[0][indeks-9][0]=='seseorang' or arr[0][indeks-9][0]=='Seseorang' or arr[0][indeks-9][0]=='Empat' or arr[0][indeks-9][0]=='Lima' or arr[0][indeks-9][0]=='Enam' or arr[0][indeks-9][0]=='Tujuh' or arr[0][indeks-9][0]=='Delapan' or arr[0][indeks-9][0]=='Sembilan' or arr[0][indeks-9][0]=='Sepuluh')):
        return arr[0][indeks-9][0]
    elif(indeks+10<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='VB' or arr[0][indeks+6][1]=='IN' or arr[0][indeks+6][1]=='NN' or arr[0][indeks+6][1]=='NNP' or arr[0][indeks+6][1]=='SC' or arr[0][indeks+6][1]=='JJ' or arr[0][indeks+6][1]=='PR'  or arr[0][indeks+6][1]=='NND'  or arr[0][indeks+6][1]=='RB') 
        and (arr[0][indeks+7][1]=='VB' or arr[0][indeks+7][1]=='IN' or arr[0][indeks+7][1]=='NN' or arr[0][indeks+7][1]=='NNP' or arr[0][indeks+7][1]=='SC' or arr[0][indeks+7][1]=='JJ' or arr[0][indeks+7][1]=='PR'  or arr[0][indeks+7][1]=='NND'  or arr[0][indeks+7][1]=='RB') 
        and (arr[0][indeks+8][1]=='VB' or arr[0][indeks+8][1]=='IN' or arr[0][indeks+8][1]=='NN' or arr[0][indeks+8][1]=='NNP' or arr[0][indeks+8][1]=='SC' or arr[0][indeks+8][1]=='JJ' or arr[0][indeks+8][1]=='PR'  or arr[0][indeks+8][1]=='NND'  or arr[0][indeks+8][1]=='RB') 
        and (arr[0][indeks+9][1]=='VB' or arr[0][indeks+9][1]=='IN' or arr[0][indeks+9][1]=='NN' or arr[0][indeks+9][1]=='NNP' or arr[0][indeks+9][1]=='SC' or arr[0][indeks+9][1]=='JJ' or arr[0][indeks+9][1]=='PR'  or arr[0][indeks+9][1]=='NND'  or arr[0][indeks+9][1]=='RB') 
        and (arr[0][indeks+10][1]=='CD' or arr[0][indeks+10][0]=='Seorang' or arr[0][indeks+10][0]=='seorang' or arr[0][indeks+10][0]=='seseorang' or arr[0][indeks+10][0]=='Seseorang' or arr[0][indeks+10][0]=='Empat' or arr[0][indeks+10][0]=='Lima' or arr[0][indeks+10][0]=='Enam' or arr[0][indeks+10][0]=='Tujuh' or arr[0][indeks+10][0]=='Delapan' or arr[0][indeks+10][0]=='Sembilan' or arr[0][indeks+10][0]=='Sepuluh')):
        return arr[0][indeks+10][0]
    elif(indeks-10>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='VB' or arr[0][indeks-6][1]=='IN' or arr[0][indeks-6][1]=='NN' or arr[0][indeks-6][1]=='NNP' or arr[0][indeks-6][1]=='SC' or arr[0][indeks-6][1]=='JJ' or arr[0][indeks-6][1]=='PR'  or arr[0][indeks-6][1]=='NND'  or arr[0][indeks-6][1]=='RB') 
        and (arr[0][indeks-7][1]=='VB' or arr[0][indeks-7][1]=='IN' or arr[0][indeks-7][1]=='NN' or arr[0][indeks-7][1]=='NNP' or arr[0][indeks-7][1]=='SC' or arr[0][indeks-7][1]=='JJ' or arr[0][indeks-7][1]=='PR'  or arr[0][indeks-7][1]=='NND'  or arr[0][indeks-7][1]=='RB') 
        and (arr[0][indeks-8][1]=='VB' or arr[0][indeks-8][1]=='IN' or arr[0][indeks-8][1]=='NN' or arr[0][indeks-8][1]=='NNP' or arr[0][indeks-8][1]=='SC' or arr[0][indeks-8][1]=='JJ' or arr[0][indeks-8][1]=='PR'  or arr[0][indeks-8][1]=='NND'  or arr[0][indeks-8][1]=='RB') 
        and (arr[0][indeks-9][1]=='VB' or arr[0][indeks-9][1]=='IN' or arr[0][indeks-9][1]=='NN' or arr[0][indeks-9][1]=='NNP' or arr[0][indeks-9][1]=='SC' or arr[0][indeks-9][1]=='JJ' or arr[0][indeks-9][1]=='PR'  or arr[0][indeks-9][1]=='NND'  or arr[0][indeks-9][1]=='RB') 
        and (arr[0][indeks-10][1]=='CD' or arr[0][indeks-10][0]=='Seorang' or arr[0][indeks-10][0]=='seorang' or arr[0][indeks-10][0]=='seseorang' or arr[0][indeks-10][0]=='Seseorang' or arr[0][indeks-10][0]=='Empat' or arr[0][indeks-10][0]=='Lima' or arr[0][indeks-10][0]=='Enam' or arr[0][indeks-10][0]=='Tujuh' or arr[0][indeks-10][0]=='Delapan' or arr[0][indeks-10][0]=='Sembilan' or arr[0][indeks-10][0]=='Sepuluh')):
        return arr[0][indeks-10][0]
    elif(indeks+11<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='VB' or arr[0][indeks+6][1]=='IN' or arr[0][indeks+6][1]=='NN' or arr[0][indeks+6][1]=='NNP' or arr[0][indeks+6][1]=='SC' or arr[0][indeks+6][1]=='JJ' or arr[0][indeks+6][1]=='PR'  or arr[0][indeks+6][1]=='NND'  or arr[0][indeks+6][1]=='RB') 
        and (arr[0][indeks+7][1]=='VB' or arr[0][indeks+7][1]=='IN' or arr[0][indeks+7][1]=='NN' or arr[0][indeks+7][1]=='NNP' or arr[0][indeks+7][1]=='SC' or arr[0][indeks+7][1]=='JJ' or arr[0][indeks+7][1]=='PR'  or arr[0][indeks+7][1]=='NND'  or arr[0][indeks+7][1]=='RB') 
        and (arr[0][indeks+8][1]=='VB' or arr[0][indeks+8][1]=='IN' or arr[0][indeks+8][1]=='NN' or arr[0][indeks+8][1]=='NNP' or arr[0][indeks+8][1]=='SC' or arr[0][indeks+8][1]=='JJ' or arr[0][indeks+8][1]=='PR'  or arr[0][indeks+8][1]=='NND'  or arr[0][indeks+8][1]=='RB') 
        and (arr[0][indeks+9][1]=='VB' or arr[0][indeks+9][1]=='IN' or arr[0][indeks+9][1]=='NN' or arr[0][indeks+9][1]=='NNP' or arr[0][indeks+9][1]=='SC' or arr[0][indeks+9][1]=='JJ' or arr[0][indeks+9][1]=='PR'  or arr[0][indeks+9][1]=='NND'  or arr[0][indeks+9][1]=='RB') 
        and (arr[0][indeks+10][1]=='VB' or arr[0][indeks+10][1]=='IN' or arr[0][indeks+10][1]=='NN' or arr[0][indeks+10][1]=='NNP' or arr[0][indeks+10][1]=='SC' or arr[0][indeks+10][1]=='JJ' or arr[0][indeks+10][1]=='PR' or arr[0][indeks+10][1]=='NND' or arr[0][indeks+10][1]=='RB') 
        and (arr[0][indeks+11][1]=='CD' or arr[0][indeks+11][0]=='Seorang' or arr[0][indeks+11][0]=='seorang' or arr[0][indeks+11][0]=='seseorang' or arr[0][indeks+11][0]=='Seseorang' or arr[0][indeks+11][0]=='Empat' or arr[0][indeks+11][0]=='Lima' or arr[0][indeks+11][0]=='Enam' or arr[0][indeks+11][0]=='Tujuh' or arr[0][indeks+11][0]=='Delapan' or arr[0][indeks+11][0]=='Sembilan' or arr[0][indeks+11][0]=='Sepuluh')):
        return arr[0][indeks+11][0]
    elif(indeks-11>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='VB' or arr[0][indeks-6][1]=='IN' or arr[0][indeks-6][1]=='NN' or arr[0][indeks-6][1]=='NNP' or arr[0][indeks-6][1]=='SC' or arr[0][indeks-6][1]=='JJ' or arr[0][indeks-6][1]=='PR'  or arr[0][indeks-6][1]=='NND'  or arr[0][indeks-6][1]=='RB') 
        and (arr[0][indeks-7][1]=='VB' or arr[0][indeks-7][1]=='IN' or arr[0][indeks-7][1]=='NN' or arr[0][indeks-7][1]=='NNP' or arr[0][indeks-7][1]=='SC' or arr[0][indeks-7][1]=='JJ' or arr[0][indeks-7][1]=='PR'  or arr[0][indeks-7][1]=='NND'  or arr[0][indeks-7][1]=='RB') 
        and (arr[0][indeks-8][1]=='VB' or arr[0][indeks-8][1]=='IN' or arr[0][indeks-8][1]=='NN' or arr[0][indeks-8][1]=='NNP' or arr[0][indeks-8][1]=='SC' or arr[0][indeks-8][1]=='JJ' or arr[0][indeks-8][1]=='PR'  or arr[0][indeks-8][1]=='NND'  or arr[0][indeks-8][1]=='RB') 
        and (arr[0][indeks-9][1]=='VB' or arr[0][indeks-9][1]=='IN' or arr[0][indeks-9][1]=='NN' or arr[0][indeks-9][1]=='NNP' or arr[0][indeks-9][1]=='SC' or arr[0][indeks-9][1]=='JJ' or arr[0][indeks-9][1]=='PR'  or arr[0][indeks-9][1]=='NND'  or arr[0][indeks-9][1]=='RB') 
        and (arr[0][indeks-10][1]=='VB' or arr[0][indeks-10][1]=='IN' or arr[0][indeks-10][1]=='NN' or arr[0][indeks-10][1]=='NNP' or arr[0][indeks-10][1]=='SC' or arr[0][indeks-10][1]=='JJ' or arr[0][indeks-10][1]=='PR' or arr[0][indeks-10][1]=='NND' or arr[0][indeks-10][1]=='RB') 
        and (arr[0][indeks-11][1]=='CD' or arr[0][indeks-11][0]=='Seorang' or arr[0][indeks-11][0]=='seorang' or arr[0][indeks-11][0]=='seseorang' or arr[0][indeks-11][0]=='Seseorang' or arr[0][indeks-11][0]=='Empat' or arr[0][indeks-11][0]=='Lima' or arr[0][indeks-11][0]=='Enam' or arr[0][indeks-11][0]=='Tujuh' or arr[0][indeks-11][0]=='Delapan' or arr[0][indeks-11][0]=='Sembilan' or arr[0][indeks-11][0]=='Sepuluh')):
        return arr[0][indeks-11][0]
    elif(indeks+12<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='VB' or arr[0][indeks+6][1]=='IN' or arr[0][indeks+6][1]=='NN' or arr[0][indeks+6][1]=='NNP' or arr[0][indeks+6][1]=='SC' or arr[0][indeks+6][1]=='JJ' or arr[0][indeks+6][1]=='PR'  or arr[0][indeks+6][1]=='NND'  or arr[0][indeks+6][1]=='RB') 
        and (arr[0][indeks+7][1]=='VB' or arr[0][indeks+7][1]=='IN' or arr[0][indeks+7][1]=='NN' or arr[0][indeks+7][1]=='NNP' or arr[0][indeks+7][1]=='SC' or arr[0][indeks+7][1]=='JJ' or arr[0][indeks+7][1]=='PR'  or arr[0][indeks+7][1]=='NND'  or arr[0][indeks+7][1]=='RB') 
        and (arr[0][indeks+8][1]=='VB' or arr[0][indeks+8][1]=='IN' or arr[0][indeks+8][1]=='NN' or arr[0][indeks+8][1]=='NNP' or arr[0][indeks+8][1]=='SC' or arr[0][indeks+8][1]=='JJ' or arr[0][indeks+8][1]=='PR'  or arr[0][indeks+8][1]=='NND'  or arr[0][indeks+8][1]=='RB') 
        and (arr[0][indeks+9][1]=='VB' or arr[0][indeks+9][1]=='IN' or arr[0][indeks+9][1]=='NN' or arr[0][indeks+9][1]=='NNP' or arr[0][indeks+9][1]=='SC' or arr[0][indeks+9][1]=='JJ' or arr[0][indeks+9][1]=='PR'  or arr[0][indeks+9][1]=='NND'  or arr[0][indeks+9][1]=='RB') 
        and (arr[0][indeks+10][1]=='VB' or arr[0][indeks+10][1]=='IN' or arr[0][indeks+10][1]=='NN' or arr[0][indeks+10][1]=='NNP' or arr[0][indeks+10][1]=='SC' or arr[0][indeks+10][1]=='JJ' or arr[0][indeks+10][1]=='PR' or arr[0][indeks+10][1]=='NND' or arr[0][indeks+10][1]=='RB') 
        and (arr[0][indeks+11][1]=='VB' or arr[0][indeks+11][1]=='IN' or arr[0][indeks+11][1]=='NN' or arr[0][indeks+11][1]=='NNP' or arr[0][indeks+11][1]=='SC' or arr[0][indeks+11][1]=='JJ' or arr[0][indeks+11][1]=='PR' or arr[0][indeks+11][1]=='NND' or arr[0][indeks+11][1]=='RB') 
        and (arr[0][indeks+12][1]=='CD' or arr[0][indeks+12][0]=='Seorang' or arr[0][indeks+12][0]=='seorang' or arr[0][indeks+12][0]=='seseorang' or arr[0][indeks+12][0]=='Seseorang' or arr[0][indeks+12][0]=='Empat' or arr[0][indeks+12][0]=='Lima' or arr[0][indeks+12][0]=='Enam' or arr[0][indeks+12][0]=='Tujuh' or arr[0][indeks+12][0]=='Delapan' or arr[0][indeks+12][0]=='Sembilan' or arr[0][indeks+12][0]=='Sepuluh')):
        return arr[0][indeks+12][0]
    elif(indeks-12>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='VB' or arr[0][indeks-6][1]=='IN' or arr[0][indeks-6][1]=='NN' or arr[0][indeks-6][1]=='NNP' or arr[0][indeks-6][1]=='SC' or arr[0][indeks-6][1]=='JJ' or arr[0][indeks-6][1]=='PR'  or arr[0][indeks-6][1]=='NND'  or arr[0][indeks-6][1]=='RB') 
        and (arr[0][indeks-7][1]=='VB' or arr[0][indeks-7][1]=='IN' or arr[0][indeks-7][1]=='NN' or arr[0][indeks-7][1]=='NNP' or arr[0][indeks-7][1]=='SC' or arr[0][indeks-7][1]=='JJ' or arr[0][indeks-7][1]=='PR'  or arr[0][indeks-7][1]=='NND'  or arr[0][indeks-7][1]=='RB') 
        and (arr[0][indeks-8][1]=='VB' or arr[0][indeks-8][1]=='IN' or arr[0][indeks-8][1]=='NN' or arr[0][indeks-8][1]=='NNP' or arr[0][indeks-8][1]=='SC' or arr[0][indeks-8][1]=='JJ' or arr[0][indeks-8][1]=='PR'  or arr[0][indeks-8][1]=='NND'  or arr[0][indeks-8][1]=='RB') 
        and (arr[0][indeks-9][1]=='VB' or arr[0][indeks-9][1]=='IN' or arr[0][indeks-9][1]=='NN' or arr[0][indeks-9][1]=='NNP' or arr[0][indeks-9][1]=='SC' or arr[0][indeks-9][1]=='JJ' or arr[0][indeks-9][1]=='PR'  or arr[0][indeks-9][1]=='NND'  or arr[0][indeks-9][1]=='RB') 
        and (arr[0][indeks-10][1]=='VB' or arr[0][indeks-10][1]=='IN' or arr[0][indeks-10][1]=='NN' or arr[0][indeks-10][1]=='NNP' or arr[0][indeks-10][1]=='SC' or arr[0][indeks-10][1]=='JJ' or arr[0][indeks-10][1]=='PR' or arr[0][indeks-10][1]=='NND' or arr[0][indeks-10][1]=='RB') 
        and (arr[0][indeks-11][1]=='VB' or arr[0][indeks-11][1]=='IN' or arr[0][indeks-11][1]=='NN' or arr[0][indeks-11][1]=='NNP' or arr[0][indeks-11][1]=='SC' or arr[0][indeks-11][1]=='JJ' or arr[0][indeks-11][1]=='PR' or arr[0][indeks-11][1]=='NND' or arr[0][indeks-11][1]=='RB') 
        and (arr[0][indeks-12][1]=='CD' or arr[0][indeks-12][0]=='Seorang' or arr[0][indeks-12][0]=='seorang' or arr[0][indeks-12][0]=='seseorang' or arr[0][indeks-12][0]=='Seseorang' or arr[0][indeks-12][0]=='Empat' or arr[0][indeks-12][0]=='Lima' or arr[0][indeks-12][0]=='Enam' or arr[0][indeks-12][0]=='Tujuh' or arr[0][indeks-12][0]=='Delapan' or arr[0][indeks-12][0]=='Sembilan' or arr[0][indeks-12][0]=='Sepuluh')):
        return arr[0][indeks-12][0]
    elif(indeks+13<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='VB' or arr[0][indeks+6][1]=='IN' or arr[0][indeks+6][1]=='NN' or arr[0][indeks+6][1]=='NNP' or arr[0][indeks+6][1]=='SC' or arr[0][indeks+6][1]=='JJ' or arr[0][indeks+6][1]=='PR'  or arr[0][indeks+6][1]=='NND'  or arr[0][indeks+6][1]=='RB') 
        and (arr[0][indeks+7][1]=='VB' or arr[0][indeks+7][1]=='IN' or arr[0][indeks+7][1]=='NN' or arr[0][indeks+7][1]=='NNP' or arr[0][indeks+7][1]=='SC' or arr[0][indeks+7][1]=='JJ' or arr[0][indeks+7][1]=='PR'  or arr[0][indeks+7][1]=='NND'  or arr[0][indeks+7][1]=='RB') 
        and (arr[0][indeks+8][1]=='VB' or arr[0][indeks+8][1]=='IN' or arr[0][indeks+8][1]=='NN' or arr[0][indeks+8][1]=='NNP' or arr[0][indeks+8][1]=='SC' or arr[0][indeks+8][1]=='JJ' or arr[0][indeks+8][1]=='PR'  or arr[0][indeks+8][1]=='NND'  or arr[0][indeks+8][1]=='RB') 
        and (arr[0][indeks+9][1]=='VB' or arr[0][indeks+9][1]=='IN' or arr[0][indeks+9][1]=='NN' or arr[0][indeks+9][1]=='NNP' or arr[0][indeks+9][1]=='SC' or arr[0][indeks+9][1]=='JJ' or arr[0][indeks+9][1]=='PR'  or arr[0][indeks+9][1]=='NND'  or arr[0][indeks+9][1]=='RB') 
        and (arr[0][indeks+10][1]=='VB' or arr[0][indeks+10][1]=='IN' or arr[0][indeks+10][1]=='NN' or arr[0][indeks+10][1]=='NNP' or arr[0][indeks+10][1]=='SC' or arr[0][indeks+10][1]=='JJ' or arr[0][indeks+10][1]=='PR' or arr[0][indeks+10][1]=='NND' or arr[0][indeks+10][1]=='RB') 
        and (arr[0][indeks+11][1]=='VB' or arr[0][indeks+11][1]=='IN' or arr[0][indeks+11][1]=='NN' or arr[0][indeks+11][1]=='NNP' or arr[0][indeks+11][1]=='SC' or arr[0][indeks+11][1]=='JJ' or arr[0][indeks+11][1]=='PR' or arr[0][indeks+11][1]=='NND' or arr[0][indeks+11][1]=='RB') 
        and (arr[0][indeks+12][1]=='VB' or arr[0][indeks+12][1]=='IN' or arr[0][indeks+12][1]=='NN' or arr[0][indeks+12][1]=='NNP' or arr[0][indeks+12][1]=='SC' or arr[0][indeks+12][1]=='JJ' or arr[0][indeks+12][1]=='PR' or arr[0][indeks+12][1]=='NND' or arr[0][indeks+12][1]=='RB') 
        and (arr[0][indeks+13][1]=='CD' or arr[0][indeks+13][0]=='Seorang' or arr[0][indeks+13][0]=='seorang' or arr[0][indeks+13][0]=='seseorang' or arr[0][indeks+13][0]=='Seseorang' or arr[0][indeks+13][0]=='Empat' or arr[0][indeks+13][0]=='Lima' or arr[0][indeks+13][0]=='Enam' or arr[0][indeks+13][0]=='Tujuh' or arr[0][indeks+13][0]=='Delapan' or arr[0][indeks+13][0]=='Sembilan' or arr[0][indeks+13][0]=='Sepuluh')):
        return arr[0][indeks+13][0]
    elif(indeks-13>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='VB' or arr[0][indeks-6][1]=='IN' or arr[0][indeks-6][1]=='NN' or arr[0][indeks-6][1]=='NNP' or arr[0][indeks-6][1]=='SC' or arr[0][indeks-6][1]=='JJ' or arr[0][indeks-6][1]=='PR'  or arr[0][indeks-6][1]=='NND'  or arr[0][indeks-6][1]=='RB') 
        and (arr[0][indeks-7][1]=='VB' or arr[0][indeks-7][1]=='IN' or arr[0][indeks-7][1]=='NN' or arr[0][indeks-7][1]=='NNP' or arr[0][indeks-7][1]=='SC' or arr[0][indeks-7][1]=='JJ' or arr[0][indeks-7][1]=='PR'  or arr[0][indeks-7][1]=='NND'  or arr[0][indeks-7][1]=='RB') 
        and (arr[0][indeks-8][1]=='VB' or arr[0][indeks-8][1]=='IN' or arr[0][indeks-8][1]=='NN' or arr[0][indeks-8][1]=='NNP' or arr[0][indeks-8][1]=='SC' or arr[0][indeks-8][1]=='JJ' or arr[0][indeks-8][1]=='PR'  or arr[0][indeks-8][1]=='NND'  or arr[0][indeks-8][1]=='RB') 
        and (arr[0][indeks-9][1]=='VB' or arr[0][indeks-9][1]=='IN' or arr[0][indeks-9][1]=='NN' or arr[0][indeks-9][1]=='NNP' or arr[0][indeks-9][1]=='SC' or arr[0][indeks-9][1]=='JJ' or arr[0][indeks-9][1]=='PR'  or arr[0][indeks-9][1]=='NND'  or arr[0][indeks-9][1]=='RB') 
        and (arr[0][indeks-10][1]=='VB' or arr[0][indeks-10][1]=='IN' or arr[0][indeks-10][1]=='NN' or arr[0][indeks-10][1]=='NNP' or arr[0][indeks-10][1]=='SC' or arr[0][indeks-10][1]=='JJ' or arr[0][indeks-10][1]=='PR' or arr[0][indeks-10][1]=='NND' or arr[0][indeks-10][1]=='RB') 
        and (arr[0][indeks-11][1]=='VB' or arr[0][indeks-11][1]=='IN' or arr[0][indeks-11][1]=='NN' or arr[0][indeks-11][1]=='NNP' or arr[0][indeks-11][1]=='SC' or arr[0][indeks-11][1]=='JJ' or arr[0][indeks-11][1]=='PR' or arr[0][indeks-11][1]=='NND' or arr[0][indeks-11][1]=='RB') 
        and (arr[0][indeks-12][1]=='VB' or arr[0][indeks-12][1]=='IN' or arr[0][indeks-12][1]=='NN' or arr[0][indeks-12][1]=='NNP' or arr[0][indeks-12][1]=='SC' or arr[0][indeks-12][1]=='JJ' or arr[0][indeks-12][1]=='PR' or arr[0][indeks-12][1]=='NND' or arr[0][indeks-12][1]=='RB') 
        and (arr[0][indeks-13][1]=='CD' or arr[0][indeks-13][0]=='Seorang' or arr[0][indeks-13][0]=='seorang' or arr[0][indeks-13][0]=='seseorang' or arr[0][indeks-13][0]=='Seseorang' or arr[0][indeks-13][0]=='Empat' or arr[0][indeks-13][0]=='Lima' or arr[0][indeks-13][0]=='Enam' or arr[0][indeks-13][0]=='Tujuh' or arr[0][indeks-13][0]=='Delapan' or arr[0][indeks-13][0]=='Sembilan' or arr[0][indeks-13][0]=='Sepuluh')):
        return arr[0][indeks-13][0]
    elif(indeks+14<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='VB' or arr[0][indeks+6][1]=='IN' or arr[0][indeks+6][1]=='NN' or arr[0][indeks+6][1]=='NNP' or arr[0][indeks+6][1]=='SC' or arr[0][indeks+6][1]=='JJ' or arr[0][indeks+6][1]=='PR'  or arr[0][indeks+6][1]=='NND'  or arr[0][indeks+6][1]=='RB') 
        and (arr[0][indeks+7][1]=='VB' or arr[0][indeks+7][1]=='IN' or arr[0][indeks+7][1]=='NN' or arr[0][indeks+7][1]=='NNP' or arr[0][indeks+7][1]=='SC' or arr[0][indeks+7][1]=='JJ' or arr[0][indeks+7][1]=='PR'  or arr[0][indeks+7][1]=='NND'  or arr[0][indeks+7][1]=='RB') 
        and (arr[0][indeks+8][1]=='VB' or arr[0][indeks+8][1]=='IN' or arr[0][indeks+8][1]=='NN' or arr[0][indeks+8][1]=='NNP' or arr[0][indeks+8][1]=='SC' or arr[0][indeks+8][1]=='JJ' or arr[0][indeks+8][1]=='PR'  or arr[0][indeks+8][1]=='NND'  or arr[0][indeks+8][1]=='RB') 
        and (arr[0][indeks+9][1]=='VB' or arr[0][indeks+9][1]=='IN' or arr[0][indeks+9][1]=='NN' or arr[0][indeks+9][1]=='NNP' or arr[0][indeks+9][1]=='SC' or arr[0][indeks+9][1]=='JJ' or arr[0][indeks+9][1]=='PR'  or arr[0][indeks+9][1]=='NND'  or arr[0][indeks+9][1]=='RB') 
        and (arr[0][indeks+10][1]=='VB' or arr[0][indeks+10][1]=='IN' or arr[0][indeks+10][1]=='NN' or arr[0][indeks+10][1]=='NNP' or arr[0][indeks+10][1]=='SC' or arr[0][indeks+10][1]=='JJ' or arr[0][indeks+10][1]=='PR' or arr[0][indeks+10][1]=='NND' or arr[0][indeks+10][1]=='RB') 
        and (arr[0][indeks+11][1]=='VB' or arr[0][indeks+11][1]=='IN' or arr[0][indeks+11][1]=='NN' or arr[0][indeks+11][1]=='NNP' or arr[0][indeks+11][1]=='SC' or arr[0][indeks+11][1]=='JJ' or arr[0][indeks+11][1]=='PR' or arr[0][indeks+11][1]=='NND' or arr[0][indeks+11][1]=='RB') 
        and (arr[0][indeks+12][1]=='VB' or arr[0][indeks+12][1]=='IN' or arr[0][indeks+12][1]=='NN' or arr[0][indeks+12][1]=='NNP' or arr[0][indeks+12][1]=='SC' or arr[0][indeks+12][1]=='JJ' or arr[0][indeks+12][1]=='PR' or arr[0][indeks+12][1]=='NND' or arr[0][indeks+12][1]=='RB') 
        and (arr[0][indeks+13][1]=='VB' or arr[0][indeks+13][1]=='IN' or arr[0][indeks+13][1]=='NN' or arr[0][indeks+13][1]=='NNP' or arr[0][indeks+13][1]=='SC' or arr[0][indeks+13][1]=='JJ' or arr[0][indeks+13][1]=='PR' or arr[0][indeks+13][1]=='NND' or arr[0][indeks+13][1]=='RB') 
        and (arr[0][indeks+14][1]=='CD' or arr[0][indeks+14][0]=='Seorang' or arr[0][indeks+14][0]=='seorang' or arr[0][indeks+14][0]=='seseorang' or arr[0][indeks+14][0]=='Seseorang' or arr[0][indeks+14][0]=='Empat' or arr[0][indeks+14][0]=='Lima' or arr[0][indeks+14][0]=='Enam' or arr[0][indeks+14][0]=='Tujuh' or arr[0][indeks+14][0]=='Delapan' or arr[0][indeks+14][0]=='Sembilan' or arr[0][indeks+14][0]=='Sepuluh')):
        return arr[0][indeks+14][0]
    elif(indeks-14>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='VB' or arr[0][indeks-6][1]=='IN' or arr[0][indeks-6][1]=='NN' or arr[0][indeks-6][1]=='NNP' or arr[0][indeks-6][1]=='SC' or arr[0][indeks-6][1]=='JJ' or arr[0][indeks-6][1]=='PR'  or arr[0][indeks-6][1]=='NND'  or arr[0][indeks-6][1]=='RB') 
        and (arr[0][indeks-7][1]=='VB' or arr[0][indeks-7][1]=='IN' or arr[0][indeks-7][1]=='NN' or arr[0][indeks-7][1]=='NNP' or arr[0][indeks-7][1]=='SC' or arr[0][indeks-7][1]=='JJ' or arr[0][indeks-7][1]=='PR'  or arr[0][indeks-7][1]=='NND'  or arr[0][indeks-7][1]=='RB') 
        and (arr[0][indeks-8][1]=='VB' or arr[0][indeks-8][1]=='IN' or arr[0][indeks-8][1]=='NN' or arr[0][indeks-8][1]=='NNP' or arr[0][indeks-8][1]=='SC' or arr[0][indeks-8][1]=='JJ' or arr[0][indeks-8][1]=='PR'  or arr[0][indeks-8][1]=='NND'  or arr[0][indeks-8][1]=='RB') 
        and (arr[0][indeks-9][1]=='VB' or arr[0][indeks-9][1]=='IN' or arr[0][indeks-9][1]=='NN' or arr[0][indeks-9][1]=='NNP' or arr[0][indeks-9][1]=='SC' or arr[0][indeks-9][1]=='JJ' or arr[0][indeks-9][1]=='PR'  or arr[0][indeks-9][1]=='NND'  or arr[0][indeks-9][1]=='RB') 
        and (arr[0][indeks-10][1]=='VB' or arr[0][indeks-10][1]=='IN' or arr[0][indeks-10][1]=='NN' or arr[0][indeks-10][1]=='NNP' or arr[0][indeks-10][1]=='SC' or arr[0][indeks-10][1]=='JJ' or arr[0][indeks-10][1]=='PR' or arr[0][indeks-10][1]=='NND' or arr[0][indeks-10][1]=='RB') 
        and (arr[0][indeks-11][1]=='VB' or arr[0][indeks-11][1]=='IN' or arr[0][indeks-11][1]=='NN' or arr[0][indeks-11][1]=='NNP' or arr[0][indeks-11][1]=='SC' or arr[0][indeks-11][1]=='JJ' or arr[0][indeks-11][1]=='PR' or arr[0][indeks-11][1]=='NND' or arr[0][indeks-11][1]=='RB') 
        and (arr[0][indeks-12][1]=='VB' or arr[0][indeks-12][1]=='IN' or arr[0][indeks-12][1]=='NN' or arr[0][indeks-12][1]=='NNP' or arr[0][indeks-12][1]=='SC' or arr[0][indeks-12][1]=='JJ' or arr[0][indeks-12][1]=='PR' or arr[0][indeks-12][1]=='NND' or arr[0][indeks-12][1]=='RB') 
        and (arr[0][indeks-13][1]=='VB' or arr[0][indeks-13][1]=='IN' or arr[0][indeks-13][1]=='NN' or arr[0][indeks-13][1]=='NNP' or arr[0][indeks-13][1]=='SC' or arr[0][indeks-13][1]=='JJ' or arr[0][indeks-13][1]=='PR' or arr[0][indeks-13][1]=='NND' or arr[0][indeks-13][1]=='RB') 
        and (arr[0][indeks-14][1]=='CD' or arr[0][indeks-14][0]=='Seorang' or arr[0][indeks-14][0]=='seorang' or arr[0][indeks-14][0]=='seseorang' or arr[0][indeks-14][0]=='Seseorang' or arr[0][indeks-14][0]=='Empat' or arr[0][indeks-14][0]=='Lima' or arr[0][indeks-14][0]=='Enam' or arr[0][indeks-14][0]=='Tujuh' or arr[0][indeks-14][0]=='Delapan' or arr[0][indeks-14][0]=='Sembilan' or arr[0][indeks-14][0]=='Sepuluh')):
        return arr[0][indeks-14][0]
    elif(indeks+15<=len(arr[0])-1 
        and (arr[0][indeks+1][1]=='VB' or arr[0][indeks+1][1]=='IN' or arr[0][indeks+1][1]=='NN' or arr[0][indeks+1][1]=='NNP' or arr[0][indeks+1][1]=='SC' or arr[0][indeks+1][1]=='JJ' or arr[0][indeks+1][1]=='PR'  or arr[0][indeks+1][1]=='NND'  or arr[0][indeks+1][1]=='RB')
        and (arr[0][indeks+2][1]=='VB' or arr[0][indeks+2][1]=='IN' or arr[0][indeks+2][1]=='NN' or arr[0][indeks+2][1]=='NNP' or arr[0][indeks+2][1]=='SC' or arr[0][indeks+2][1]=='JJ' or arr[0][indeks+2][1]=='PR'  or arr[0][indeks+2][1]=='NND'  or arr[0][indeks+2][1]=='RB') 
        and (arr[0][indeks+3][1]=='VB' or arr[0][indeks+3][1]=='IN' or arr[0][indeks+3][1]=='NN' or arr[0][indeks+3][1]=='NNP' or arr[0][indeks+3][1]=='SC' or arr[0][indeks+3][1]=='JJ' or arr[0][indeks+3][1]=='PR'  or arr[0][indeks+3][1]=='NND'  or arr[0][indeks+3][1]=='RB') 
        and (arr[0][indeks+4][1]=='VB' or arr[0][indeks+4][1]=='IN' or arr[0][indeks+4][1]=='NN' or arr[0][indeks+4][1]=='NNP' or arr[0][indeks+4][1]=='SC' or arr[0][indeks+4][1]=='JJ' or arr[0][indeks+4][1]=='PR'  or arr[0][indeks+4][1]=='NND'  or arr[0][indeks+4][1]=='RB') 
        and (arr[0][indeks+5][1]=='VB' or arr[0][indeks+5][1]=='IN' or arr[0][indeks+5][1]=='NN' or arr[0][indeks+5][1]=='NNP' or arr[0][indeks+5][1]=='SC' or arr[0][indeks+5][1]=='JJ' or arr[0][indeks+5][1]=='PR'  or arr[0][indeks+5][1]=='NND'  or arr[0][indeks+5][1]=='RB') 
        and (arr[0][indeks+6][1]=='VB' or arr[0][indeks+6][1]=='IN' or arr[0][indeks+6][1]=='NN' or arr[0][indeks+6][1]=='NNP' or arr[0][indeks+6][1]=='SC' or arr[0][indeks+6][1]=='JJ' or arr[0][indeks+6][1]=='PR'  or arr[0][indeks+6][1]=='NND'  or arr[0][indeks+6][1]=='RB') 
        and (arr[0][indeks+7][1]=='VB' or arr[0][indeks+7][1]=='IN' or arr[0][indeks+7][1]=='NN' or arr[0][indeks+7][1]=='NNP' or arr[0][indeks+7][1]=='SC' or arr[0][indeks+7][1]=='JJ' or arr[0][indeks+7][1]=='PR'  or arr[0][indeks+7][1]=='NND'  or arr[0][indeks+7][1]=='RB') 
        and (arr[0][indeks+8][1]=='VB' or arr[0][indeks+8][1]=='IN' or arr[0][indeks+8][1]=='NN' or arr[0][indeks+8][1]=='NNP' or arr[0][indeks+8][1]=='SC' or arr[0][indeks+8][1]=='JJ' or arr[0][indeks+8][1]=='PR'  or arr[0][indeks+8][1]=='NND'  or arr[0][indeks+8][1]=='RB') 
        and (arr[0][indeks+9][1]=='VB' or arr[0][indeks+9][1]=='IN' or arr[0][indeks+9][1]=='NN' or arr[0][indeks+9][1]=='NNP' or arr[0][indeks+9][1]=='SC' or arr[0][indeks+9][1]=='JJ' or arr[0][indeks+9][1]=='PR'  or arr[0][indeks+9][1]=='NND'  or arr[0][indeks+9][1]=='RB') 
        and (arr[0][indeks+10][1]=='VB' or arr[0][indeks+10][1]=='IN' or arr[0][indeks+10][1]=='NN' or arr[0][indeks+10][1]=='NNP' or arr[0][indeks+10][1]=='SC' or arr[0][indeks+10][1]=='JJ' or arr[0][indeks+10][1]=='PR' or arr[0][indeks+10][1]=='NND' or arr[0][indeks+10][1]=='RB') 
        and (arr[0][indeks+11][1]=='VB' or arr[0][indeks+11][1]=='IN' or arr[0][indeks+11][1]=='NN' or arr[0][indeks+11][1]=='NNP' or arr[0][indeks+11][1]=='SC' or arr[0][indeks+11][1]=='JJ' or arr[0][indeks+11][1]=='PR' or arr[0][indeks+11][1]=='NND' or arr[0][indeks+11][1]=='RB') 
        and (arr[0][indeks+12][1]=='VB' or arr[0][indeks+12][1]=='IN' or arr[0][indeks+12][1]=='NN' or arr[0][indeks+12][1]=='NNP' or arr[0][indeks+12][1]=='SC' or arr[0][indeks+12][1]=='JJ' or arr[0][indeks+12][1]=='PR' or arr[0][indeks+12][1]=='NND' or arr[0][indeks+12][1]=='RB') 
        and (arr[0][indeks+13][1]=='VB' or arr[0][indeks+13][1]=='IN' or arr[0][indeks+13][1]=='NN' or arr[0][indeks+13][1]=='NNP' or arr[0][indeks+13][1]=='SC' or arr[0][indeks+13][1]=='JJ' or arr[0][indeks+13][1]=='PR' or arr[0][indeks+13][1]=='NND' or arr[0][indeks+13][1]=='RB') 
        and (arr[0][indeks+14][1]=='VB' or arr[0][indeks+14][1]=='IN' or arr[0][indeks+14][1]=='NN' or arr[0][indeks+14][1]=='NNP' or arr[0][indeks+14][1]=='SC' or arr[0][indeks+14][1]=='JJ' or arr[0][indeks+14][1]=='PR' or arr[0][indeks+14][1]=='NND' or arr[0][indeks+14][1]=='RB') 
        and (arr[0][indeks+15][1]=='CD' or arr[0][indeks+15][0]=='Seorang' or arr[0][indeks+15][0]=='seorang' or arr[0][indeks+15][0]=='seseorang' or arr[0][indeks+15][0]=='Seseorang' or arr[0][indeks+15][0]=='Empat' or arr[0][indeks+15][0]=='Lima' or arr[0][indeks+15][0]=='Enam' or arr[0][indeks+15][0]=='Tujuh' or arr[0][indeks+15][0]=='Delapan' or arr[0][indeks+15][0]=='Sembilan' or arr[0][indeks+15][0]=='Sepuluh')):
        return arr[0][indeks+15][0]
    elif(indeks-15>=0 
        and (arr[0][indeks-1][1]=='VB' or arr[0][indeks-1][1]=='IN' or arr[0][indeks-1][1]=='NN' or arr[0][indeks-1][1]=='NNP' or arr[0][indeks-1][1]=='SC' or arr[0][indeks-1][1]=='JJ' or arr[0][indeks-1][1]=='PR'  or arr[0][indeks-1][1]=='NND'  or arr[0][indeks-1][1]=='RB')
        and (arr[0][indeks-2][1]=='VB' or arr[0][indeks-2][1]=='IN' or arr[0][indeks-2][1]=='NN' or arr[0][indeks-2][1]=='NNP' or arr[0][indeks-2][1]=='SC' or arr[0][indeks-2][1]=='JJ' or arr[0][indeks-2][1]=='PR'  or arr[0][indeks-2][1]=='NND'  or arr[0][indeks-2][1]=='RB') 
        and (arr[0][indeks-3][1]=='VB' or arr[0][indeks-3][1]=='IN' or arr[0][indeks-3][1]=='NN' or arr[0][indeks-3][1]=='NNP' or arr[0][indeks-3][1]=='SC' or arr[0][indeks-3][1]=='JJ' or arr[0][indeks-3][1]=='PR'  or arr[0][indeks-3][1]=='NND'  or arr[0][indeks-3][1]=='RB') 
        and (arr[0][indeks-4][1]=='VB' or arr[0][indeks-4][1]=='IN' or arr[0][indeks-4][1]=='NN' or arr[0][indeks-4][1]=='NNP' or arr[0][indeks-4][1]=='SC' or arr[0][indeks-4][1]=='JJ' or arr[0][indeks-4][1]=='PR'  or arr[0][indeks-4][1]=='NND'  or arr[0][indeks-4][1]=='RB') 
        and (arr[0][indeks-5][1]=='VB' or arr[0][indeks-5][1]=='IN' or arr[0][indeks-5][1]=='NN' or arr[0][indeks-5][1]=='NNP' or arr[0][indeks-5][1]=='SC' or arr[0][indeks-5][1]=='JJ' or arr[0][indeks-5][1]=='PR'  or arr[0][indeks-5][1]=='NND'  or arr[0][indeks-5][1]=='RB') 
        and (arr[0][indeks-6][1]=='VB' or arr[0][indeks-6][1]=='IN' or arr[0][indeks-6][1]=='NN' or arr[0][indeks-6][1]=='NNP' or arr[0][indeks-6][1]=='SC' or arr[0][indeks-6][1]=='JJ' or arr[0][indeks-6][1]=='PR'  or arr[0][indeks-6][1]=='NND'  or arr[0][indeks-6][1]=='RB') 
        and (arr[0][indeks-7][1]=='VB' or arr[0][indeks-7][1]=='IN' or arr[0][indeks-7][1]=='NN' or arr[0][indeks-7][1]=='NNP' or arr[0][indeks-7][1]=='SC' or arr[0][indeks-7][1]=='JJ' or arr[0][indeks-7][1]=='PR'  or arr[0][indeks-7][1]=='NND'  or arr[0][indeks-7][1]=='RB') 
        and (arr[0][indeks-8][1]=='VB' or arr[0][indeks-8][1]=='IN' or arr[0][indeks-8][1]=='NN' or arr[0][indeks-8][1]=='NNP' or arr[0][indeks-8][1]=='SC' or arr[0][indeks-8][1]=='JJ' or arr[0][indeks-8][1]=='PR'  or arr[0][indeks-8][1]=='NND'  or arr[0][indeks-8][1]=='RB') 
        and (arr[0][indeks-9][1]=='VB' or arr[0][indeks-9][1]=='IN' or arr[0][indeks-9][1]=='NN' or arr[0][indeks-9][1]=='NNP' or arr[0][indeks-9][1]=='SC' or arr[0][indeks-9][1]=='JJ' or arr[0][indeks-9][1]=='PR'  or arr[0][indeks-9][1]=='NND'  or arr[0][indeks-9][1]=='RB') 
        and (arr[0][indeks-10][1]=='VB' or arr[0][indeks-10][1]=='IN' or arr[0][indeks-10][1]=='NN' or arr[0][indeks-10][1]=='NNP' or arr[0][indeks-10][1]=='SC' or arr[0][indeks-10][1]=='JJ' or arr[0][indeks-10][1]=='PR' or arr[0][indeks-10][1]=='NND' or arr[0][indeks-10][1]=='RB') 
        and (arr[0][indeks-11][1]=='VB' or arr[0][indeks-11][1]=='IN' or arr[0][indeks-11][1]=='NN' or arr[0][indeks-11][1]=='NNP' or arr[0][indeks-11][1]=='SC' or arr[0][indeks-11][1]=='JJ' or arr[0][indeks-11][1]=='PR' or arr[0][indeks-11][1]=='NND' or arr[0][indeks-11][1]=='RB') 
        and (arr[0][indeks-12][1]=='VB' or arr[0][indeks-12][1]=='IN' or arr[0][indeks-12][1]=='NN' or arr[0][indeks-12][1]=='NNP' or arr[0][indeks-12][1]=='SC' or arr[0][indeks-12][1]=='JJ' or arr[0][indeks-12][1]=='PR' or arr[0][indeks-12][1]=='NND' or arr[0][indeks-12][1]=='RB') 
        and (arr[0][indeks-13][1]=='VB' or arr[0][indeks-13][1]=='IN' or arr[0][indeks-13][1]=='NN' or arr[0][indeks-13][1]=='NNP' or arr[0][indeks-13][1]=='SC' or arr[0][indeks-13][1]=='JJ' or arr[0][indeks-13][1]=='PR' or arr[0][indeks-13][1]=='NND' or arr[0][indeks-13][1]=='RB') 
        and (arr[0][indeks-14][1]=='VB' or arr[0][indeks-14][1]=='IN' or arr[0][indeks-14][1]=='NN' or arr[0][indeks-14][1]=='NNP' or arr[0][indeks-14][1]=='SC' or arr[0][indeks-14][1]=='JJ' or arr[0][indeks-14][1]=='PR' or arr[0][indeks-14][1]=='NND' or arr[0][indeks-14][1]=='RB') 
        and (arr[0][indeks-15][1]=='CD' or arr[0][indeks-15][0]=='Seorang' or arr[0][indeks-15][0]=='seorang' or arr[0][indeks-15][0]=='seseorang' or arr[0][indeks-15][0]=='Seseorang' or arr[0][indeks-15][0]=='Empat' or arr[0][indeks-15][0]=='Lima' or arr[0][indeks-15][0]=='Enam' or arr[0][indeks-15][0]=='Tujuh' or arr[0][indeks-15][0]=='Delapan' or arr[0][indeks-15][0]=='Sembilan' or arr[0][indeks-15][0]=='Sepuluh')):
        return arr[0][indeks-15][0]
    else:
        return 1
    

    #dan (CC) limitasi untuk sementara 

In [9]:
def indeksKe(arr,indikator):
    for x in range(0, len(arr[0])):
        for y in indikator:
            if(arr[0][x][0]==y):
                return x
    return -1

In [10]:
def convertTeks(teks):
    if(teks=="Satu" or teks=="satu" or teks=="Seorang" or teks=="Seseorang" or teks=="Sesosok" or teks=="seorang" or teks=="seseorang" or teks=="sesosok"):
        return 1
    elif(teks=="Dua" or teks=="dua" or teks=="Kedua" or teks=="kedua"):
        return 2
    elif(teks=="Tiga" or teks=="tiga" or teks=="Ketiga" or teks=="ketiga"):
        return 3
    elif(teks=="Empat" or teks=="empat" or teks=="Keempat" or teks=="keempat"):
        return 4
    elif(teks=="Lima" or teks=="lima" or teks=="Kelima" or teks=="kelima"):
        return 5
    elif(teks=="Enam" or teks=="enam" or teks=="Keenam" or teks=="keenam"):
        return 6
    elif(teks=="Tujuh" or teks=="tujuh" or teks=="Ketujuh" or teks=="ketujuh"):
        return 7
    elif(teks=="Delapan" or teks=="delapan" or teks=="Kedelapan" or teks=="kedelapan"):
        return 8
    elif(teks=="Sembilan" or teks=="sembilan" or teks=="Kesembilan" or teks=="kesembilan"):
        return 9
    elif(teks=="Sepuluh" or teks=="sepuluh" or teks=="Kesepuluh" or teks=="kesepuluh"):
        return 10
    else:
        return teks

In [11]:
from datetime import datetime, timedelta

month_to_number = {
  'jan': '01',
  'feb': '02',
  'mar': '03',
  'apr': '04',
  'mei': '05',
  'jun': '06',
  'jul': '07',
  'agu': '08',
  'sep': '09',
  'okt': '10',
  'nov': '11',
  'des': '12',
}

def getNewsTime(timestamp: str, news: str, source: str):
  if source == "www.detik.com":
    splitted_timestamp = timestamp.split()
    pre_formatted_time = splitted_timestamp[3] + '-' + month_to_number[splitted_timestamp[2].lower()] + '-' + splitted_timestamp[1] + ' ' + splitted_timestamp[4] + ':00'

    date_time_obj = datetime.strptime(pre_formatted_time, '%Y-%m-%d %H:%M:%S')
    
    # 0 sabtu, 1 05, 2 Mar, 3 2022, 4 15:33, 5 WIB
    if "kemarin" in news:
      yesterday = date_time_obj - timedelta(1)
      return yesterday.strftime('%Y-%m-%d %H:%M:%S')
    elif "selumbari" in news:
      daysago = date_time_obj - timedelta(2)
      return daysago.strftime('%Y-%m-%d %H:%M:%S')
    else:
      return date_time_obj

In [12]:
def bow_read(bow):
    bow_read = pd.read_csv(bow)
    bow_text = []
    # print(bow_read.shape[0])

    for i in range(0,bow_read.shape[0]):
        if(bow_read.iloc[i,1] not in bow_text and bow_read.iloc[i,1]!=0):
            bow_text.append(bow_read.iloc[i,1])
        
    return(bow_read,bow_text)

In [13]:
bow_kecelakaan_read,bow_kecelakaan_text = bow_read('bow_kecelakaan.csv')

In [14]:
print(len(bow_kecelakaan_text))

5579


In [15]:
def cari_dokpertama(kueriAsli):
    kueriPre=preprocessingCorpus(kueriAsli)
    kueriPre= " ".join (kueriPre)
    hasilSearch=[]
    tfidf_train = joblib.load('tfidf_train.pkl')
    tfidf_vectorizer = joblib.load('vectorizer.pkl')
    
    query_vec= tfidf_vectorizer.transform([kueriPre])

    results=cosine_similarity(tfidf_train, query_vec).reshape((-1))
    for i in results.argsort()[-5:][::-1]:
        hasilSearch.append(df_train.iloc[i,-2])
    hasilSearch=". ".join(hasilSearch)
    return hasilSearch



In [16]:
##Keywords Extraction with YAKE
def keyword_yake(hasilSearch):
    keywordYake=[]

    k_extractor = KeywordExtractor(lan="id", n=1, top=50)
    k_extractor2 = KeywordExtractor(lan="id", n=2, top=50)

    keywords = k_extractor.extract_keywords(text=hasilSearch)
    keywords = k_extractor2.extract_keywords(text=hasilSearch)

    keywordYake = [x for x, y in keywords]
    return keywordYake

In [17]:
#Keywords Extraction with TFIDF
def keyword_tfidf(hasilSearch):

    keywordtfidf=[]
    keywordtfidf2=[]

    #doc = 'بَاب فرض الْوضُوء وسننه وهيآته وَفرض الْوضُوء سِتّ خِصَال النِّيَّة عمند غسل الْوَجْه وَغسل الْوَجْه وَغسل الذراعين مَعَ الْمرْفقين وَمسح مَا قل من الرَّأْس وَغسل الرجلَيْن مَعَ الْكَعْبَيْنِ وَالتَّرْتِيب وعَلى قَول الْوَلَاء وسننه عشر خِصَال خمس مِنْهَا قبل غسل الْوَجْه وَهِي التَّسْمِيَة وَغسل الْكَفَّيْنِ والمضمضة وَالِاسْتِنْشَاق وَالْمُبَالغَة فيههما إِلَّا للصَّائِم وَخمْس بعد غسل الْوَجْه وَهِي تَقْدِيم الْيُمْنَى على ليسرى وَمسح جَمِيع الرَّأْس وَمسح الْأُذُنَيْنِ ظاهرهما وباطنهما وَإِدْخَال الأصبعين فيهمَا وتخليل أَصَابِع الرجلَيْن . وَغسل دَاخل الْكَعْبَيْنِ وَلَيْسَ مسح لعنق من سنَنه وفضيلته تكراره ثَلَاثًا وزالواجب فِيهِ مرّة والمرتان أفضل وَالثَّلَاث أكمل وهيآته أَن يبْدَأ فِي تَطْهِير الْأَعْضَاء بمواضع الِابْتِدَاء . فَإِن اقْتصر على فروضه استة أَجزَأَهُ وَإِن ضيع حَظّ نَفسه فِيمَا ترك'
    total_words = re.sub(r'[^\w]', ' ', hasilSearch)
    total_words = total_words.lower().split()
    total_word_length = len(total_words)
    total_sentences = tokenize.sent_tokenize(hasilSearch)
    total_sent_len = len(total_sentences)

    tf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('.','')
        if each_word not in NLTK_StopWords:
            if each_word in tf_score:
                tf_score[each_word] += 1
            else:
                tf_score[each_word] = 1

    # Dividing by total_word_length for each dictionary element
    tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
    def check_sent(word, sentences): 
        final = [all([w in x for w in word]) for x in sentences] 
        sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
        return int(len(sent_len))

    idf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('.','')
        if each_word not in NLTK_StopWords:
            if each_word in idf_score:
                idf_score[each_word] = check_sent(each_word, total_sentences)
            else:
                idf_score[each_word] = 1

    # Performing a log and divide
    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}

    def get_top_n(dict_elem, n):
        result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
        hasil =list(result.keys()) 
        return hasil
    keywordtfidf.append(get_top_n(tf_idf_score, 35))
    for i in range(len(keywordtfidf)):
        totalKw=0
        totalKw=len(keywordtfidf[i])
        for j in range(totalKw):
            keywordtfidf2.append(keywordtfidf[i][j])
    return keywordtfidf2

In [18]:
#Keywords Extraction with BERT
def keyword_bert(hasilSearch):

    keywordbert=[]

    keyword1 = kw_extractor.extract_keywords(hasilSearch, top_n=50, keyphrase_ngram_range=(1, 1))
    keyword2 = kw_extractor.extract_keywords(hasilSearch, top_n=50, keyphrase_ngram_range=(1, 2))

    for i in range (0,len (keyword1)):
        keywordbert.append(keyword1[i][0])
        keywordbert.append(keyword2[i][0])
    return keywordbert

In [19]:
def rangking (keywordGabung):
    keywordTemp=[]
    keywordFinal=[]

    def borda_sort(lists):
        scores = {}
        for l in lists:
            for idx, elem in enumerate(reversed(l)):
                if not elem in scores:
                    scores[elem] = 0
                scores[elem] += idx
        return sorted(scores.keys(), key=lambda elem: scores[elem], reverse=True)

    keywordTemp.append(borda_sort(keywordGabung))
    print ('kandidat temp',keywordTemp)
    print ('Total Kandidat temp: ',len(keywordTemp[0]))

    if len(keywordTemp[0])>30:
        print ('kurang dari 80')
        for i in range (0,80):
            keywordFinal.append(keywordTemp[0][i])
    elif len(keywordTemp[0])<80:
        print ('lebih dari 80')
        for i in range (0,len(keywordTemp)):
            for j in range (0,len(keywordTemp[0])):
                keywordFinal.append(keywordTemp[0][j])
    print ('Total Kandidat final: ',len(keywordFinal))
    print ('Kandidat final: ',keywordFinal)
    return keywordFinal


In [20]:
def keyword_BOW(keywordBOW, kueriAsli):
    cekDuplicate = []
    kandidatFix = []

    for i in keywordBOW:
        if(i not in cekDuplicate and i!=0):
            cekDuplicate.append(i)

    queries=kueriAsli
    query_embeddings = embedder.encode(queries)
    corpus_embeddings4 = embedder.encode(cekDuplicate)
    
    # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
    closest_n = 2200
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings4, 'cosine')[0]
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        for idx, distance in results[0:closest_n]:
            kandidatFix.append(cekDuplicate[idx])
    return kandidatFix

In [21]:
def kandidatFix(kueriAsli,bow):
    #kueri what
    # kueriAsli='apa sebenarnya kejadian yang terjadi diberita tersebut'
    kueri=preprocessingCorpus(kueriAsli)
    kueri= [" ".join (kueri)]
    print (kueri)
    hasilkandidat=[]
    keywordGabung=[]
    kandidatFix=[]
    kueriFix=[]
    kueriFixWithDelimiter=[]
    


    hasilSearch=cari_dokpertama(kueriAsli)
    keywordYake=keyword_yake(hasilSearch)
    keywordtfidf2=keyword_tfidf(hasilSearch)
    keywordbert=keyword_bert (hasilSearch)
    keywordBOW=keyword_BOW(bow, kueri)

    keywordGabung.append(keywordYake)
    keywordGabung.append(keywordtfidf2)
    keywordGabung.append(keywordbert)
    # keywordGabung.append(keywordBOW)
    hasilrank=rangking(keywordGabung)

    for i in hasilrank:
        kueriFix.append(i)
    for x in keywordBOW:
        kueriFix.append(x)
    for j in kueriFix:
        hasilkandidat.append(j)
    kueriFixWithDelimiter=kueriFix
    kueriFix=[preprocessingCorpus(i) for i in kueriFix]
    for i in kueriFix:
        for j in i:
            kandidatFix.append(j)
    kandidatFix= [" ".join (kandidatFix)]
    
    print ('*'*120)
    return(kandidatFix,keywordGabung,keywordBOW,hasilrank,kueriFixWithDelimiter)


In [22]:
provinsi= joblib.load('provinsi.pkl')
kotaKab= joblib.load('kotaKab.pkl')
kecamatan= joblib.load('kecamatan.pkl')

In [23]:
# hasilDokumenWhen=[]
# hasilDokumenWho=[]
# hasilDokumenWhere=[]
# hasilDokumenWhat=[]

kueriAsliWhat='apa sebenarnya kejadian kecelakaan yang terjadi diberita tersebut'
# kueriAsliWhat='musibah apa yang terjadi'
# kueriAsliWhere='daerah lokasi tempat terjadinya kecelakaan'
# kueriAsliWhere='kota kejadian terjadi'
# kueriAsliWho='siapa pelaku kejadian ini'
# kueriAsliWhen='hari apa waktu terjadinya'

kandidatFixWhat,keywordGabungWhat,keywordBOW_What, hasilrankWhat, kueriFixWithDelimiter_What = kandidatFix(kueriAsliWhat, bow_kecelakaan_text)
# kandidatFixWhere,keywordGabungWhere,keywordBOW_Where, hasilrankWhere,kueriFixWithDelimiter_Where = kandidatFix(kueriAsliWhere, bow_where_text)
# kandidatFixWho,keywordGabungWho,keywordBOW_Who, hasilrankWho, kueriFixWithDelimiter_Who = kandidatFix(kueriAsliWho, bow_who_text)
# kandidatFixWhen,keywordGabungWhen,keywordBOW_When, hasilrankWhen, kueriFixWithDelimiter_When = kandidatFix(kueriAsliWhen, bow_when_text)



['kejadian kecelakaan diberita']
kandidat temp [['almarhumah sri', 'almarhumah menangis', 'almarhumah', 'keluarga almarhumah', 'jenazah almarhumah', 'almarhumah menyambut', 'keluarga', 'jatim keluarga', 'jatiroto lumajang', 'kedungmulyo', '1264', 'jatiroto', 'kejauhan', 'warga kejauhan', 'penghubung', 'pucangsimo', 'dikemudikan tubagus', 'kemutakhiran', 'kecamatan jatiroto', 'harapannya almarhumah', 'kabupaten', 'dibopong keluarga', 'dikemudikan', 'langsung dibawa', 'pagerwojo', 'almarhumah khusnul', 'pemulangan jenazah', 'sidoarjo meninggalnya', 'kelurahan', '1264 bju', 'jenazah', 'hijau', 'pagerwojo kecamatan', 'almarhumah reva', 'kekuatan', 'melayu', 'langsung', 'meninggalnya', 'terjatuh', 'limbangan warga', 'ketahui', 'warga kelurahan', 'tersangkut', 'limbangan tutup', 'kemanusiaan', 'keberadaan', 'desa pagerwojo', 'tubagus', 'hartatik', '61 almarhumah', 'hidayatullah', 'jenazah dikeluarkan', 'kedungmulyo jombang', 'jenazah sri', 'terowongan', 'penanganan pmi', 'masyarakat membutuh

In [24]:
def cekLokation(arr,location):
    hasil=[]
    for x in arr:
        for y in location:
            if(x==y and x not in hasil):
                hasil.append(x)

    return hasil


In [26]:
def Calculation():
    document_result=[]
    document_result_what=[]
    # document_result_provinsi=[]
    
    tfidf_vectorizer = joblib.load('vectorizer.pkl')
    
    for i in range(0, len(df_test)):
        #What
        hasilWhat=[]
        hasilProv=[]
        hasilKabKot=[]
        hasilKec=[]
        trueWhat=[]

        corpus_teks1=preprocessingCorpus(df_test.iloc[i,-2])
        corpus_teks = (' '.join(corpus_teks1)) 
        tfidf_matrix = tfidf_vectorizer.fit_transform([corpus_teks])

        query_vec_What= tfidf_vectorizer.transform(kandidatFixWhat)
        results_what=cosine_similarity(tfidf_matrix, query_vec_What).reshape((-1))
        document_result_what.append(corpus_teks)
        for a in kueriFixWithDelimiter_What:
            cariW = re.findall(a,document_result_what[i])
            #print(cariW)
            if cariW:
                hasilWhat.append(a)
        
        for x in hasilWhat:
            if((x=='kecelakaan' or x=='tertabrak' or x=='tabrakan' or x=='pesawat jatuh' or x=='kapal' or x=='tenggelam' or x=='terbalik')and x not in trueWhat):
                trueWhat.append(x)
        
        #where
        # for a in provinsi:
        #     prov = re.findall(a,document_result_what[i])
        #     if (prov and prov not in hasilProv):
        #         hasilProv.append(a)
        # for a in kotaKab:
        #     kabKot = re.findall(a,document_result_what[i])
        #     if (kabKot and kabKot not in hasilKabKot):
        #         hasilKabKot.append(a)
        # for a in kecamatan:
        #     kec = re.findall(a,document_result_what[i])
        #     if (kec and kec not in hasilKec):
        #         hasilKec.append(a)


        # kabKotFix = cekLokation(hasilKabKot,corpus_teks1)
        # kecamatanFix = cekLokation(hasilKec,corpus_teks1)
        

        #When
        date = df_test.iloc[i, -3]
        source = df_test.iloc[i, -1]
        extracted_time = getNewsTime(timestamp=date, news=corpus_teks, source=source)

        #keparahan
        arr =[] 
        keparahan_teks =preprocessingKeparahan(df_test.iloc[i,-2])
        arr.append(keparahan_teks)
        tag_keparahan = ct.tag_sents(arr)

        death_indeks= indeksKe(tag_keparahan,['meninggal','mati','tewas','nyawa','menewaskan']) #kematian #terseret
        injury_indeks= indeksKe(tag_keparahan,['luka-luka','luka','melukai','terluka'])# korban,evakuasi
        crash_indeks= indeksKe(tag_keparahan,['menabrak'])# korban,evakuasi

        if(death_indeks==-1):
            mati = 0
        else:
            mati = struktur1(tag_keparahan,death_indeks)
            if(isinstance(mati, str)):
                mati=convertTeks(mati)

        if(injury_indeks==-1):
            luka = 0
        else:
            luka = struktur1(tag_keparahan,injury_indeks)
            if(isinstance(luka, str)):
                luka=convertTeks(luka)

        if(crash_indeks==-1):
            menabrak = 0
        else:
            menabrak = struktur1(tag_keparahan,crash_indeks)
            if(isinstance(menabrak, str)):
                menabrak=convertTeks(menabrak)
        
        # data = [i,df_test.iloc[i,2],trueWhat, extracted_time, mati, luka,menabrak,hasilProv,hasilKabKot,hasilKec]
        data = [i,df_test.iloc[i,2],trueWhat, hasilWhat, extracted_time, mati, luka,menabrak]
        document_result.append(data)

    #write to csv
    writer = pd.DataFrame(document_result, columns=['No Document','Description', 'What', 'hasilWhat','When', 'Mati', 'Luka','menabrak'])
    writer.to_csv('QE_Stat_testing_result1_6.csv', index=False, sep=',')

In [27]:
Calculation()